# ------------- PROYECTO FINAL G & S--------------------

## LIBRERIAS

In [ ]:
import cv2 #librería para manejo de imágenes y video
import numpy as np #librería para manejo de arreglos y matrices
import mediapipe as mp #librería para detección de gestos y poses
import pickle #librería para serializar objetos de Python
import os #librería para manejar archivos y directorios
import time #librería para manejar el tiempo

# Librerías para manejo de datos
import tensorflow as tf #librería para manejo de redes neuronales y aprendizaje profundo
from tensorflow.keras.models import Sequential, load_model #librería para manejo de modelos de Keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization #librería para manejo de capas de Keras
from tensorflow.keras.optimizers import Adam #librería para manejo de optimizadores de Keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau #librería para manejo de callbacks de Keras 
from tensorflow.keras.regularizers import l2 #librería para manejo de regularizadores de Keras

# Librerías para manejo de datos tabulares
from sklearn.preprocessing import StandardScaler, LabelEncoder #librería para manejo de escaladores y codificadores de etiquetas
from sklearn.model_selection import train_test_split #librería para manejo de división de datos en conjuntos de entrenamiento y prueba
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report #librería para manejo de métricas de clasificación

# Comunicación y cámara
import socket #librería para manejo de sockets y comunicación en red
import queue #librería para manejo de colas

# Voz
import pyttsx3 # librería para síntesis de voz
import threading # librería para manejo de hilos

# Módulos para reconocimiento de voz
import speech_recognition as sr # librería para reconocimiento de voz
import librosa # librería para análisis de audio
import io # librería para manejo de flujos de entrada/salida
import wave # librería para manejo de archivos de audio en formato WAV

# Generador de audios
from pydub import AudioSegment # librería para manejo de archivos de audio
from pydub.silence import detect_nonsilent # librería para detección de silencios en archivos de audio

## UDP

In [ ]:
# Configuración para el reconocimiento de voz
SAMPLE_RATE_IN = 48000  # Tasa de muestreo del micrófono INMP441 en Hz
SAMPLE_RATE_OUT = 16000  # Tasa de muestreo requerida por la API de reconocimiento de voz (Google, etc.)
BUFFER_DURATION = 5  # Duración en segundos del buffer de audio que se acumula antes de enviar (5 s)


UDP_IP_PI = "192.168.7.2"  # IP de la Raspberry Pi donde corre el receptor UDP
UDP_OPEN = '0.0.0.0' # IP local de enlace para recibir en cualquier interfaz

# Puertos para diferentes servicios

UDP_PORT_SERVO = 5001      # envío de comandos de ángulo al servo
UDP_PORT_PARLANTE = 5003   # recepción de audio TTS para reproducir
UDP_PORT_CAM = 5002        # envío/recepción de fragmentos de vídeo
UDP_PORT_MICROFONO = 5006  # envío de audio capturado por micrófono
UDP_PORT_TEXT = 5005       # recepción de texto transcrito (API → PC)

# Agregar al inicio del archivo
UDP_PORT_HANDS_STATUS = 5007  # Nuevo puerto
sock_hands_status = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# ---- Agregar al inicio del archivo ----
UDP_PORT_HEARTBEAT = 5008
sock_heartbeat = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# MSS
MAX_PACKET_SIZE = 1460  # Tamaño máximo en bytes de cada paquete UDP (MTU 1500 – cabeceras)

# Buffer UDP
BUFFER_UDP = 65536 # Tamaño máximo del buffer de recepción UDP en bytes (~64 KiB)

1.	SAMPLE_RATE_IN = 48000
Define a qué frecuencia (en muestras por segundo) el micrófono INMP441 está capturando audio. Un valor alto (48 kHz) mejora la calidad de captura.
2.	SAMPLE_RATE_OUT = 16000
Muchas APIs de reconocimiento de voz (por ejemplo Google Speech-to-Text) requieren audio a 16 kHz. Este parámetro se usa para re-muestrear el buffer antes de enviarlo.
3.	BUFFER_DURATION = 5
Indica que el sistema acumulará 5 segundos de audio en un buffer antes de re-muestrear y enviar todo el bloque a la API. Controla la latencia de transcripción.
4.	UDP_IP_PI = "192.168.7.2"
Dirección IP fija de la Raspberry Pi embebida. El script de recolección envía ahí el audio y otros comandos.
5.	UDP_OPEN = '0.0.0.0'
Al enlazar un socket receptor, se usa esta IP para que escuche en todas las interfaces de red disponibles del host.
6.	UDP_PORT_MICROFONO = 5006
Puerto UDP donde el script envía bloques de audio PCM originados en el micrófono.
7.	UDP_PORT_TEXT = 5005
Puerto UDP donde el script recibe las cadenas de texto transcrito que debe mostrar en pantalla.
8.	UDP_PORT_SERVO = 5001
Puerto UDP utilizado para enviar desde el PC los ángulos a los que debe moverse el servomotor de las gafas.
9.	UDP_PORT_PARLANTE = 5003
Puerto UDP donde la Raspberry Pi recibe el audio TTS (voz sintetizada) enviado por el PC tras reconocimiento de gesto.
10.	UDP_PORT_CAM = 5002
Puerto UDP donde se fragmentan y envían los frames de vídeo codificados por la Pi hacia el PC.
11.	UDP_PORT_HANDS_STATUS = 5007
Define un nuevo puerto UDP (5007) dedicado a recibir desde la PC el estado de detección de manos (“HANDS_DETECTED” o “NO_HANDS”). Esto permite que la Raspberry Pi sepa si debe activar o desactivar el servo o la lógica de visualización en función de si hay una mano visible.
12.	sock_hands_status = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
Crea un socket UDP (AF_INET y SOCK_DGRAM) que se enlazará al puerto anterior para recibir mensajes de estado de manos. No se bloquea, para poder integrar su lectura en el bucle principal o en un select.
13.	UDP_PORT_HEARTBEAT = 5008
Establece otro puerto UDP (5008) para recibir señales de latido (“heartbeat”) periódicas desde la PC. Estas señales garantizan que la conexión sigue viva; si deja de llegar el heartbeat, la Pi asume que la PC o la conexión falló y puede desactivar componentes por seguridad.
14.	sock_heartbeat = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
Crea un socket UDP para recibir los mensajes de heartbeat. Al igual que con sock_hands_status, se usa AF_INET y SOCK_DGRAM y se integrará en el mismo mecanismo de lectura no bloqueante (por ejemplo, usando select) junto con los otros sockets.
15.	MAX_PACKET_SIZE = 1460
Ajusta el tamaño máximo útil de datos por paquete UDP para no fragmentar a nivel IP (1500 MTU – 20 bytes IP – 20 bytes UDP).
16.	BUFFER_UDP = 65536
Reserva 64 KiB para el buffer de sockets UDP, permitiendo recibir paquetes grandes o ráfagas antes de desbordar.


In [ ]:
# Función para enviar heartbeat
def send_heartbeat(): #Declara la función send_heartbeat que se encargará de enviar pulsos (“heartbeats”) periódicos a la Raspberry Pi.
    while True: #Inicia un bucle infinito, de modo que se envíe un heartbeat cada intervalo definido.
        try:
            sock_heartbeat.sendto(b"HEARTBEAT", (UDP_IP_PI, UDP_PORT_HEARTBEAT)) #Usa el socket UDP sock_heartbeat para enviar el mensaje de bytes b"HEARTBEAT" a la dirección (UDP_IP_PI, UDP_PORT_HEARTBEAT) (la IP y el puerto de la Pi). Esto informa al receptor de que el enlace todavía está activo.
            time.sleep(1) #función durante 1 segundo, controlando la frecuencia de envío (1 Hz).
        except Exception as e:
            break  # Salir si hay error

# Iniciar hilo de heartbeat al comenzar el programa
heartbeat_thread = threading.Thread(target=send_heartbeat, daemon=True) #Crea un hilo de ejecución (Thread) que ejecutará la función send_heartbeat. El parámetro daemon=True indica que este hilo no bloqueará la finalización del programa; se cerrará automáticamente cuando el programa principal termine.
heartbeat_thread.start() #Arranca el hilo, que inmediatamente comenzará a ejecutar send_heartbeat() en paralelo al resto del script.

Con esto, el sistema garantiza que, mientras el programa principal esté corriendo, se enviará un heartbeat cada segundo a la Raspberry Pi. 

## CAMARA

### MEDIAPIPE

In [ ]:
# Inicializar MediaPipe
mp_hands = mp.solutions.hands #Accede al módulo de MediaPipe especializado en manos. A partir de aquí puedes invocar clases y funciones para detección y seguimiento de landmarks de la mano
hands = mp_hands.Hands( #Crea una instancia del detector de manos con los siguientes parámetros:
    static_image_mode=False, #Indica que el modelo espera vídeo continuo. Si fuese True, trataría cada frame como imagen independiente (más lento).
    max_num_hands=2, #Número máximo de manos que intentará detectar en cada frame. Aquí se deja en 2 para cubrir casos de gestos con ambas manos.
    min_detection_confidence=0.5, #Umbral mínimo (0.0–1.0) de confianza para reportar una detección de mano. Con 0.5 se filtran detecciones débiles; bajar a 0.4 aumentaría sensibilidad (más falsos positivos).
    min_tracking_confidence=0.5 #Umbral mínimo para continuar rastreando una mano ya detectada entre frames. Un valor bajo (p. ej. 0.4) permite que el tracker mantenga seguimiento aún con oclusiones parciales.
)
mp_drawing = mp.solutions.drawing_utils #Obtiene el módulo de utilidades gráficas de MediaPipe. Con él puedes dibujar en el frame los landmarks y conexiones de la mano (esqueleto)

### COMUNICACION CAMARA

Usada para recibir imágenes en tiempo real enviadas por la Raspberry Pi mediante paquetes UDP fragmentados

Permite recibir fragmentos de imagen JPEG enviados por UDP desde la Raspberry Pi, reconstruirlos, y entregarlos como un frame OpenCV utilizable.

In [ ]:
class UDPCamera:
    def __init__(self):
        self.host = UDP_OPEN #Escucha en todas las interfaces de red (IP 0.0.0.0).
        self.port = UDP_PORT_CAM #Puerto donde se reciben los fragmentos de imagen enviados por la cámara (Raspberry Pi).
        self.buffer_size = BUFFER_UDP #Define el tamaño del buffer de recepción de paquetes. Típicamente 65536 bytes.
        self.mtu = MAX_PACKET_SIZE #Máximo tamaño de fragmento que puede recibir sin que se fragmente a nivel IP (normalmente 1460 bytes).
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) #Crea el socket UDP que se usará para escuchar los fragmentos de imagen.
        self.sock.settimeout(2) #Establece un tiempo máximo de espera (2 segundos) para que recvfrom no bloquee indefinidamente.
        self.frame = None #Contendrá el último frame completo decodificado en formato OpenCV.
        self.fragments = [] #Lista temporal que almacena los fragmentos UDP de una imagen hasta que estén todos.
        self.running = False #Bandera booleana que controla si el hilo de recepción está activo.
        self.thread = None #Referencia al hilo que recibe paquetes en segundo plano.
        self.lock = threading.Lock() #Lock para sincronizar acceso entre hilos a self.frame y self.fragments.
        self.start() #Llama al método start() automáticamente al construir la clase.

    def start(self):
        if not self.running: #Evita que se inicie el hilo más de una vez.
            self.running = True #Asocia el socket al host y puerto definidos para que escuche los paquetes entrantes.
            self.sock.bind((self.host, self.port)) #Crea un hilo nuevo que ejecutará la función _receive_frames.
            self.thread = threading.Thread(target=self._receive_frames, daemon=True) #Asegura que el hilo se cierra automáticamente al terminar el programa principal.
            self.thread.start() #Inicia el hilo que estará escuchando los fragmentos de imagen.

    def _receive_frames(self): # Función que se ejecuta en un hilo separado para recibir los fragmentos de imagen.
        while self.running: # Bucle que se ejecuta mientras self.running sea True.
            try:
                fragment, _ = self.sock.recvfrom(self.buffer_size) # Recibe un fragmento de imagen (máximo 1460 bytes).
                with self.lock:
                    self.fragments.append(fragment) #Almacena el fragmento en la lista temporal.
                    if len(fragment) < self.mtu:  # Si el fragmento recibido es más pequeño que el máximo permitido, se asume que es el último fragmento de la imagen.
                        frame_bytes = b''.join(self.fragments) #Une todos los fragmentos acumulados en un único bloque binario.
                        self.fragments = [] # Limpia la lista de fragmentos para recibir una nueva imagen.
                        frame_array = np.frombuffer(frame_bytes, dtype=np.uint8) #Convierte el bloque binario a un array NumPy de bytes.
                        self.frame = cv2.imdecode(frame_array, cv2.IMREAD_COLOR) #Decodifica la imagen JPEG para obtener un frame válido de OpenCV.

            except socket.timeout:
                continue
            except Exception as e:
                print(f"Error en recepción: {str(e)}")
                break

    def read(self):
        with self.lock:
            if self.frame is not None:
                return True, self.frame.copy()
            return False, None
    """ 	Si hay uno válido, devuelve (True, frame)
            Si no, devuelve (False, None)
            copy() asegura que no se modifique accidentalmente el frame mientras otro hilo lo lee."""

    def release(self):
        self.running = False
        with self.lock:
            self.fragments = []
            self.frame = None
        if self.thread and self.thread.is_alive():
            self.thread.join(timeout=1)
        self.sock.close()

        """ Marca running como falso → el hilo terminará.
            Limpia frame y fragments.
            Espera que el hilo se cierre.
            Cierra el socket UDP. """


    def __del__(self):
        self.release() #Destructor de la clase. Asegura que al eliminar la instancia, todos los recursos se limpien correctamente (buen diseño).

### MODELO TFLITE

Encargada de cargar y ejecutar el modelo optimizado en TensorFlow Lite:


In [ ]:
#Define una clase para encapsular la carga y ejecución de un modelo TensorFlow Lite.
class TFLiteModel:
    def __init__(self, model_path): #Constructor que recibe la ruta al archivo .tflite del modelo optimizado
        
        # Cargar el modelo TFLite
        self.interpreter = tf.lite.Interpreter(model_path=model_path) #Crea un intérprete de TensorFlow Lite cargando los pesos y la arquitectura desde el fichero model_path.
        self.interpreter.allocate_tensors() #Reserva memoria para todos los tensores de entrada y salida del modelo en el intérprete. Debe llamarse antes de cualquier inferencia.
        
        # Obtener detalles de entrada y salida
        self.input_details = self.interpreter.get_input_details() #Recupera metadatos sobre las entradas del modelo: nombre, índice, forma esperada (shape) y tipo de dato (dtype).
        self.output_details = self.interpreter.get_output_details() #De manera análoga, recupera metadatos sobre las salidas del modelo.
    
    def predict(self, input_data): #Método público para hacer una inferencia dado un vector de características input_data.
        # Asegurar el tipo de dato correcto y agregar dimensión batch si es necesario
        input_data = np.array(input_data, dtype=self.input_details[0]['dtype']) #Convierte la entrada a un array NumPy con el tipo de dato que el modelo espera (por ejemplo float32).
        #Comprobar y ajustar batch dimension
        if len(input_data.shape) == len(self.input_details[0]['shape']) - 1:
            input_data = np.expand_dims(input_data, axis=0)
        """o	Si input_data viene como (n_features,) (una sola muestra), esta condición detecta que 
        falta la dimensión de batch y la añade con expand_dims, convirtiéndolo en (1, n_features)."""

        # Establecer la entrada y ejecutar la inferencia
        self.interpreter.set_tensor(self.input_details[0]['index'], input_data) #Copia los datos de input_data al tensor de entrada interno del intérprete, usando el índice que corresponde a la primera (y única) entrada.
        self.interpreter.invoke() #Ejecuta la inferencia: el intérprete procesa el grafo TFLite usando los tensores de entrada ya cargados, y produce valores en los tensores de salida.
        
        # Obtener la salida
        output_data = self.interpreter.get_tensor(self.output_details[0]['index']) #Recupera los datos del tensor de salida (por ejemplo, el vector de probabilidades resultante de la capa softmax), usando el índice de salida.
        return output_data #Devuelve el array NumPy con la salida de la inferencia, que luego se procesa (por ejemplo, con np.argmax) para obtener la clase predicha.


Con esta clase, el código principal puede crear instancias así:

model = TFLiteModel('modelo_optimizadotl_v15.tflite')

probs = model.predict(feature_vector)

predicted_class = np.argmax(probs)

y de esta forma aislar toda la complejidad de TensorFlow Lite en un envoltorio muy sencillo.


In [ ]:
# Configuración de TensorFlow para rendimiento
physical_devices = tf.config.list_physical_devices('GPU') #Consulta a TensorFlow si hay dispositivos físicos del tipo GPU disponibles en el sistema. Devuelve una lista de objetos Device.
if physical_devices: #Si la lista no está vacía (existen GPUs), entra al bloque.
    # Configuración de TensorFlow para rendimiento en CPU
    try:
        # Verificar si hay GPU disponible (para futuras expansiones)
        physical_devices = tf.config.list_physical_devices('GPU') #Vuelve a obtener la lista de GPUs (repetición intencional o por comodidad de contexto).
        
        if physical_devices: #Confirma nuevamente la presencia de GPUs.
            # Configuración para GPU (no se ejecutará en tu caso)
            for device in physical_devices: #Itera sobre cada GPU detectada.
                tf.config.experimental.set_memory_growth(device, True) #Activa el crecimiento dinámico de memoria en GPU: TensorFlow reservará solo lo necesario en lugar de toda la memoria GPU al inicio.
            print("GPU disponible para aceleración")
        else:
            # Optimización para CPU
            tf.config.threading.set_intra_op_parallelism_threads(4)  # Aprovecha núcleos físicos
            tf.config.threading.set_inter_op_parallelism_threads(2)  # Paralelismo entre operaciones
            print("Modo CPU activado: Configuración optimizada para Intel Core i7-7500U")
            
    except Exception as e:
        print(f"Error de configuración: {str(e)}")
        print("Usando configuración por defecto de CPU")

Con esto, el script ajusta TensorFlow para maximizar rendimiento en función del hardware disponible, sin bloquear la ejecución si algo falla.

### ARCHIVOS

In [ ]:
# Configuración de directorios y archivos
data_dir = "hand_gestures_data_v15" #Define la ruta (nombre de carpeta) donde se almacenarán todos los archivos relacionados con la recolección de datos y el entrenamiento.
os.makedirs(data_dir, exist_ok=True) #Crea la carpeta hand_gestures_data_v15 si no existe. exist_ok=True evita lanzar excepción si la carpeta ya está creada.

# Modelo y datos de entrenamiento
model = None #Inicializa la variable que contendrá el modelo Keras. Se asignará más adelante una instancia de Sequential tras definir la arquitectura.
# Inicializar scaler y label encoder
scaler = StandardScaler() #Crea un objeto StandardScaler vacío, que luego se ajustará (fit) sobre los datos de entrenamiento para calcular medias y desviaciones.
label_encoder = LabelEncoder() #Crea un objeto LabelEncoder que traducirá las etiquetas de texto (nombres de seña) a índices numéricos.
model_file = "hand_gesture_nn_model_v15.h5"  #Nombre de archivo donde se guardará el modelo Keras entrenado en formato HDF5.
scaler_file = "hand_gesture_scaler_v15.pkl" #Ruta donde se serializará el objeto StandardScaler con pickle.
encoder_file = "hand_gesture_encoder_v15.pkl" #Archivo para el objeto LabelEncoder serializado.
gesture_data = "gesture_data_v15.pkl"  #Nombre del archivo .pkl que contendrá el dataset completo de vectores de características y etiquetas reunidos.
model_tflite = "modelo_optimizadotl_v15.tflite" #Ruta del archivo de salida para el modelo convertido a TensorFlow Lite.


De este modo, el script centraliza en variables todos los nombres de carpetas y archivos, facilitando su mantenimiento y evitando literales repartidos por el código.
Aquí tienes la explicación línea por línea y por bloques del fragmento que define variables globales en el script me_15_v6_recoleccion.ipynb. Estas variables son esenciales para controlar el estado de la recolección, el entrenamiento y la evaluación del modelo de reconocimiento de gestos.


In [ ]:
# Variables globales para estado
data = [] #Lista vacía donde se almacenarán los vectores de características (arrays NumPy) extraídos de los landmarks de las manos (coordenadas, distancias, ángulos).
labels = [] #Lista paralela a data que guarda las etiquetas textuales asociadas a cada vector (por ejemplo "HOLA", "GRACIAS").

# Estado del sistema
is_trained = False #Bandera booleana que indica si el modelo ya fue entrenado. Se usa para habilitar su uso en tiempo real o en pruebas.
is_collecting = False #Flag que determina si actualmente se están recogiendo muestras. Se activa cuando el usuario presiona una tecla para empezar a grabar un gesto específico.
current_gesture = "" #Cadena que almacena el nombre de la seña actualmente en recolección. Ejemplo: "COMO", "AYUDA".
samples_collected = 0 #Contador de cuántas muestras se han recolectado para la seña actual. Se reinicia al iniciar la recolección de una nueva seña.
max_samples = 5000 #Número máximo de muestras que se permite recolectar en total o por clase, dependiendo de cómo esté implementada la lógica de corte.

# Control de tiempo para la recolección continua
last_sample_time = 0 #Guarda la última vez (en segundos desde epoch) que se agregó una muestra. Se compara contra time.time() para espaciar capturas.
sample_delay = 0.05  # Tiempo mínimo entre muestras: 0.05 s = 50 ms.
#Evita capturar demasiadas muestras en poco tiempo (por ejemplo si el gesto no cambia), y mejora la variedad dentro de una clase.


# Temporizador para mostrar mensajes
message = "" #Texto que se muestra en pantalla como retroalimentación al usuario (ej: “Recolectando ‘HOLA’...”, “Modelo entrenado con éxito”, etc.).
message_until = 0 #Marca de tiempo (en segundos) hasta la cual se mostrará el mensaje. Pasado este tiempo, el mensaje desaparece automáticamente del display.

# Para evaluación del modelo
metrics = {
    'accuracy': 0,
    'val_accuracy': 0,
    'training_time': 0
}
"""Diccionario que guarda los resultados de entrenamiento del modelo:
o	'accuracy': precisión final en el conjunto de entrenamiento
o	'val_accuracy': precisión sobre el conjunto de validación
o	'training_time': duración total del proceso de entrenamiento (en segundos)
"""

### EXTRACCION DE LANDMARKS

In [ ]:
last_hands_detected_time = 0 #Guarda el timestamp (segundos desde epoch) de la última vez que se detectó al menos una mano, para gestionar estados “sin manos”.
hands_detected_status = False #Flag que indica si actualmente hay manos siendo detectadas (True) o no (False).

In [ ]:
send_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # Crea un socket UDP para enviar datos (en este caso, la coordenada normalizada de la muñeca derecha al servo en la Pi).

def extract_hand_landmarks(frame, send_sock=None): #Declara la función que recibe un frame (imagen BGR) y opcionalmente un socket send_sock para enviar coordenadas.
    global last_hands_detected_time, hands_detected_status
    """
    Extrae landmarks de las manos y realiza seguimiento de la muñeca derecha.
    
    Args:
        frame: Imagen en formato BGR
        send_sock: Socket UDP opcional para enviar datos de seguimiento
    
    Returns:
        tuple: (landmarks_data, hands_detected)
    """
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #Convierte el frame de BGR (OpenCV) a RGB (formato requerido por MediaPipe).
    results = hands.process(frame_rgb) #Pasa el frame a MediaPipe Hands para obtener results.multi_hand_landmarks y results.multi_handedness.
    
    landmarks_data = [] #Lista temporal para acumular valores [x,y,z] de cada landmark de ambas manos (hasta 2×21×3 = 126 valores).
    hands_detected = False #Flag local que se pondrá a True si se detecta al menos una mano.
    x_normalized = None #Variable para almacenar la coordenada normalizada de la muñeca derecha en rango [−7.5,7.5][-7.5,7.5].
    right_wrist_pixel = None #Coordenadas en píxeles de la muñeca derecha para dibujar un círculo.
    
    if results.multi_hand_landmarks: #Comprueba si MediaPipe detectó alguna mano en el frame.
        hands_detected = True #Marca que hay manos para este frame.
        # Procesar ambas manos para landmarks
        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks): #Itera sobre cada mano detectada (hasta max_num_hands).
            # Dibujar landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS) #Dibuja en el frame los puntos y conexiones de la mano para visual feedback.
            
            # Extraer coordenadas
            landmarks = [] #Lista temporal para esta mano.
            for landmark in hand_landmarks.landmark: #Recorre los 21 landmarks de la mano.
                landmarks.extend([landmark.x, landmark.y, landmark.z])  #Añade las tres coordenadas normalizadas de cada landmark.
            landmarks_data.extend(landmarks) #Concatena los valores de esta mano al buffer global landmarks_data.
            
            # Detectar mano derecha para seguimiento
            if results.multi_handedness: #Comprueba si se proporcionó información de lateralidad (izquierda/derecha).
                handedness = results.multi_handedness[hand_idx] #Obtiene el resultado de clasificación para esta mano.
                if handedness.classification[0].label == 'Left' and not x_normalized: #Aquí ‘Left’ corresponde a la mano DERECHA del usuario (configuración invertida). Se asegura de procesar solo la primera detección.
                    wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST] #Toma el landmark de la muñeca.
                    
                    # Calcular coordenadas normalizadas
                    x_normalized = int((wrist.x - 0.5) * 15)  # Rango -7.5 a 7.5,Normaliza la coordenada X de [0,1][0,1] a [−7.5,7.5][-7.5,7.5]. Este valor controla el ángulo del servomotor.
                    #print(x_normalized)
                    
                    # Obtener coordenadas para dibujo
                    right_wrist_pixel = mp_drawing._normalized_to_pixel_coordinates( #Convierte la posición normalizada (wrist.x,wrist.y)(wrist.x, wrist.y) al sistema de píxeles del frame (width, height).
                        wrist.x, wrist.y, frame.shape[1], frame.shape[0]
                    )

        # Enviar datos y dibujar si se detectó mano derecha
        if x_normalized is not None: #Solo si se detectó muñeca derecha, envía y dibuja:
            if send_sock:  # Solo enviar si se provee socket, Verifica que el socket se haya pasado como argumento.
                send_sock.sendto(  #Envía la coordenada normalizada como bytes al puerto de servo de la Pi.
                    str(x_normalized).encode(),
                    (UDP_IP_PI, UDP_PORT_SERVO)  # Asegurar que estas constantes están definidas
                )
            if right_wrist_pixel:
                cv2.circle(frame, right_wrist_pixel, 10, (0, 255, 0), -1) #Dibuja un círculo verde de radio 10 píxeles en la posición de la muñeca.
    
    # Rellenar con ceros si no hay manos
    while len(landmarks_data) < 21 * 3 * 2:
        landmarks_data.append(0.0)
    landmarks_data = landmarks_data[:21 * 3 * 2]
    """ o	Cada frame debe devolver siempre 126 valores (2 manos × 21 landmarks × 3 coordenadas).
        o	Si no se detectaron manos o faltan valores, se añaden ceros hasta completar.
        o	Luego se recorta con [:126] para asegurar tamaño exacto.
    """

        # Enviar estado de detección
    current_time = time.time()
    if hands_detected:
        sock_hands_status.sendto(b"HANDS_DETECTED", (UDP_IP_PI, UDP_PORT_HANDS_STATUS)) #notifica inmediatamente al puerto de estado.
        last_hands_detected_time = current_time #actualiza el último instante detectado.
        hands_detected_status = True #marca estado global.
    else:
        """o	Comprueba si ha pasado más de 1 s desde la última detección.
           o	Entonces envía b"NO_HANDS" y pone hands_detected_status = False.
        """
        if current_time - last_hands_detected_time > 1:  # Esperar 2 segundos sin manos
            sock_hands_status.sendto(b"NO_HANDS", (UDP_IP_PI, UDP_PORT_HANDS_STATUS))
            hands_detected_status = False
    
    return landmarks_data, hands_detected #Devuelve el vector fijado en longitud y un flag booleano indicando si se detectaron manos.

In [42]:
def set_message(message_text, duration=2):
    global message, message_until
    message = message_text
    message_until = time.time() + duration

### RECOLECCION

In [ ]:
def start_collection(gesture_name): #Se define la función start_collection que recibe como parámetro gesture_name, el nombre de la seña que el usuario va a recoger.
    global is_collecting, current_gesture, samples_collected
    is_collecting = True #Activa el modo de recolección. A partir de este momento, el bucle principal del script empezará a capturar y almacenar muestras de gestos.
    current_gesture = gesture_name #Guarda en la variable global el nombre de la seña que se va a recolectar. Esto se usa luego para etiquetar cada vector de características con la etiqueta correcta.
    samples_collected = 0 #Reinicia el contador de muestras recolectadas a cero, para comenzar a contar desde el primer frame etiquetado en esta sesión de recolección.
    set_message(f"Mantenga la seña frente a la cámara. Recolectando '{gesture_name}'...", 3)

def stop_collection():#Define la función stop_collection sin parámetros, que detiene la recolección en curso. 
    global is_collecting, current_gesture, samples_collected
    is_collecting = False #Desactiva el modo de recolección. El bucle principal dejará de capturar y almacenar muestras.
    current_gesture = "" #Limpia el nombre de la seña actual, ya que ya no se está recolectando ninguna.
    samples_collected = 0 #Reinicia el contador de muestras (opcional, para evitar residuos en próximas colecciones).
    set_message("Recolección finalizada", 2)

Estas dos funciones se llaman desde el bucle principal al detectar la pulsación de teclas específicas (por ejemplo, ord('a') para empezar la seña "A") y al pulsar la tecla de parada (q o similar). Controlan todo el flujo de recolección de datos y garantizan que cada muestra se etiquete correctamente.

### GUARDADO DE DATOS

In [ ]:
def save_data(): #Declara la función save_data sin parámetros, destinada a almacenar en disco las muestras y etiquetas recogidas hasta el momento.
    global data, labels
    data_to_save = {"features": data, "labels": labels}
    """Crea un diccionario que agrupa las dos listas en un único objeto:
        o	"features": contendrá todos los vectores de características (data)
        o	"labels": las etiquetas correspondientes (labels)
    Esto facilita la carga posterior, manteniendo juntos datos y etiquetas.
    """

    with open(f"{data_dir}/{gesture_data}", "wb") as f:
        """Abre (o crea) el archivo de salida en modo binario ("wb") dentro del directorio data_dir con el nombre definido en gesture_data 
        (p.ej. "gesture_data_v15.pkl"). Usa un bloque with para asegurar el cierre automático del archivo."""
        pickle.dump(data_to_save, f) #Serializa el diccionario data_to_save y lo escribe en el archivo abierto. De este modo se guarda todo el dataset de manera persistente.
    set_message(f"Datos guardados: {len(data)} muestras", 1)

### RECOLECCION DE MUESTRAS

In [ ]:
def collect_sample(landmarks): #Define una función que recibe como parámetro landmarks, un vector de características (por ejemplo, 126 valores extraídos de los puntos de MediaPipe + distancias + ángulos) correspondiente a un frame.
    global is_collecting, samples_collected, last_sample_time, data, labels
    
    if not is_collecting: #i no estamos en modo recolección, abortamos la función inmediatamente y devolvemos False, ya que no corresponde capturar nada.
        return False
    
    current_time = time.time() #Captura el timestamp actual (en segundos desde epoch) para controlar la frecuencia de muestreo.
    if current_time - last_sample_time >= sample_delay: #5.	Comprueba si ha pasado el tiempo mínimo entre capturas (por defecto 50 ms). Esto evita capturar muestras demasiado rápido que sean casi idénticas.
        data.append(landmarks) #agrega el vector actual a la lista global de muestras.
        labels.append(current_gesture) #guarda la etiqueta textual correspondiente a esta muestra.

        #Incrementa el contador de muestras recolectadas y actualiza el tiempo del último muestreo exitoso.
        samples_collected += 1
        last_sample_time = current_time
        
        if samples_collected % max_samples == 0:
            save_data()
        """Si se ha llegado a un múltiplo de max_samples, se llama a save_data() para persistir lo recolectado.
        (Aunque este uso parece redundante con el bloque siguiente: probablemente está pensado por seguridad si no se usa stop_collection.)
        """


        if samples_collected >= max_samples:
            stop_collection()
            return True
        """Si se alcanzó el número máximo de muestras para esta seña:
        o	Se detiene la recolección automáticamente con stop_collection().
        o	Devuelve True para avisar al flujo principal que se completó la captura.
        """
    
    return False #Si no se cumplen las condiciones para guardar o finalizar, retorna False, lo que indica que la captura continúa.

¿Qué hace en sí esta función?

•Controla la recolección temporizada de vectores de gestos.

•Guarda cada muestra con su etiqueta correspondiente.

•Evita sobrecargar el dataset con datos repetidos por estar demasiado próximos en el tiempo.

•Finaliza automáticamente cuando se completa el cupo por gesto.

•Permite al flujo principal mantenerse simple: solo tiene que llamar continuamente collect_sample(...) y revisar si devolvió True.


### CARGA DE DATOS

In [ ]:
def load_data(): #Declara la función load_data sin parámetros, que intentará leer el dataset serializado de gestos desde el sistema de archivos.
    global data, labels
    try:
        with open(f"{data_dir}/{gesture_data}", "rb") as f:
            """Abre el archivo cuyo nombre combina la ruta data_dir y el nombre gesture_data (por ejemplo 
            "hand_gestures_data_v15/gesture_data_v15.pkl") en modo de lectura binaria ("rb"). 
            El bloque with garantiza el cierre automático del archivo."""
             
            loaded_data = pickle.load(f) #Usa pickle para deserializar el contenido del archivo. Se espera que sea un diccionario con las claves "features" y "labels".
            data = loaded_data["features"] #Extrae la lista de vectores de características del diccionario y la asigna a la variable global data.
            labels = loaded_data["labels"] #Extrae la lista de etiquetas correspondientes y la asigna a la variable global labels.
        set_message(f"Datos cargados: {len(data)} muestras", 2)
        return True
    except Exception as e:
        print(f"Error al cargar datos: {e}")
        set_message("No se encontraron datos previos", 2)
        return False

Con esto, la función load_data permite reutilizar datasets previos, acelerando el inicio del entrenamiento o evitando recolectar muestras desde cero cada vez.

### RED NEURONAL

In [ ]:
def check_model_exists():
    return os.path.exists(model_file) and os.path.exists(scaler_file) and os.path.exists(encoder_file)


def create_neural_network(input_shape, num_classes):
    """Parámetros:
    •	input_shape: cantidad de características que recibe cada vector de entrada (por ejemplo, 126 si usás 2 manos × 21 puntos × 3 coordenadas).
    •	num_classes: cantidad total de señas diferentes que querés reconocer.
    """
    #Se define un modelo secuencial, es decir, una pila lineal de capas conectadas una tras otra (sin bifurcaciones).
    model = Sequential([  
        Dense(64, activation='relu', input_shape=(input_shape,), kernel_regularizer=l2(0.001)), 
        BatchNormalization(),
        Dropout(0.3), 
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(), 
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001), #Algoritmo de optimización adaptativo. Aprende más rápido y de forma más estable que SGD.
        #Tamaño de paso del optimizador. Ni muy grande (inestable), ni muy chico (lento).
        loss='sparse_categorical_crossentropy', #Función de pérdida para clasificación multiclase con etiquetas enteras. Se aplica cuando las clases se codifican como 0, 1, 2, ..., y no como one-hot.
        metrics=['accuracy'] #Mide qué porcentaje de predicciones coincide con la etiqueta real.
    )
    
    return model

```python
    model = Sequential([  
        Dense(64, activation='relu', input_shape=(input_shape,), kernel_regularizer=l2(0.001)), 
        """ •	Dense(64): Crea una capa completamente conectada con 64 neuronas.
            •	activation='relu': Cada neurona aplica la función ReLU(x) = max(0, x)
                    Introduce no linealidad y ayuda a que la red aprenda relaciones complejas.
            •	input_shape=(input_shape,): Define la dimensión de entrada.
                    Por ejemplo, si cada vector tiene 126 características, entonces recibe una tupla (126,).
            •	kernel_regularizer=l2(0.001): Aplica regularización L2 (también llamada “weight decay”) para penalizar pesos grandes y evitar el sobreajuste:
            el calculo seria: perdida total=cross-entropy + lamda*sum(w^2), en este caso lamda=0.001.
            """

        BatchNormalization(),
        """ •	Esta capa normaliza las salidas de la capa anterior (media ≈ 0, varianza ≈ 1) para estabilizar y acelerar el entrenamiento.
            •	Calcula, por cada mini-batch, media y desviación estándar de las salidas de las 64 neuronas anteriores.
            •	También permite que la red aprenda una escala y un sesgo nuevos, si es útil.
            """
      
        Dropout(0.3),
        """ •	Desactiva aleatoriamente el 30% de las neuronas de esta capa durante el entrenamiento.
            •	Evita que la red dependa demasiado de ciertas neuronas (sobreajuste) y fuerza a que aprenda representaciones más generales.
            """
        
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        """ •	Capa totalmente conectada de 32 neuronas con activación ReLU.
            •	También tiene regularización L2.
            •	Actúa como transformación intermedia para reducir la dimensionalidad y extraer patrones más abstractos.
            """
        BatchNormalization(), #Aplica normalización a las 32 neuronas.

        Dropout(0.2), #desactiva aleatoriamente el 20% de ellas en cada batch de entrenamiento.

        Dense(num_classes, activation='softmax')
        """ •	Capa de salida con tantas neuronas como clases (señas distintas).
            •	activation='softmax' transforma las salidas en probabilidades:
                yk=e^zk / Σe^zi
                donde zk es la salida de la k-ésima neurona y Σe^zi es la suma de todas las exponenciales de las salidas.
            •	Esto asegura que todas las salidas sumen 1, permitiendo interpretarlas como probabilidades de pertenencia a cada clase.
            •   Cada neurona da la probabilidad estimada de que el vector de entrada corresponda a una seña específica.
                """
    ])

¿Cómo funciona en conjunto?
1.	Entrás con un vector de características numéricas (por ejemplo, 126 valores).
2.	Las capas densas lo transforman y lo comprimen, buscando patrones en los movimientos de las manos.
3.	Se regulariza y normaliza para que aprenda sin sobreajustarse.
4.	La capa softmax da una probabilidad para cada seña conocida.
5.	El modelo aprende ajustando los pesos para que la salida se acerque cada vez más a la etiqueta correcta.


### ENTRENAMIENTO

In [ ]:
def train_model():
    global model, scaler, label_encoder, metrics, is_trained
    """ o	model: contendrá la red entrenada.
        o	scaler: el objeto StandardScaler.
        o	label_encoder: el LabelEncoder.
        o	metrics: diccionario donde guardamos precisión y tiempo.
        o	is_trained: flag que indica si ya hay un modelo listo.
        """
    
    #Validación mínima de datos
    if len(data) < 10:
        set_message("Se necesitan más datos para entrenar", 2)
        return False
    """
        o	Si hemos recolectado menos de 10 muestras en total, mostramos un mensaje (“Se necesitan más datos...”) y abortamos el entrenamiento devolviendo False.
        o	Esto evita errores de shape y de entrenamiento sobre conjuntos demasiado pequeños.
        """
    #Construcción de matrices de entrada y salida
    X = np.array(data) #array de forma (N, F), con N muestras y F características (landmarks+distancias+ángulos).
    y = np.array(labels) #vector de forma (N,) con etiquetas textuales para cada muestra.
    
    # Codificar etiquetas
    y_encoded = label_encoder.fit_transform(y)
    """ o	fit_transform ajusta internamente el LabelEncoder al conjunto y, generando un array de enteros y_encoded en {0,...,C-1}.
        o	La asociación texto→índice se guarda en label_encoder.classes_.
    """
    
    # Dividir datos
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    """ o 	test_size=0.2: reserva el 20% de las muestras para evaluación final.
        o	random_state=42: semilla fija para reproducibilidad.
        o	stratify=y_encoded: asegura que la proporción de cada clase en train/test sea la misma, evitando sesgo si algunas señas son minoritarias.
        """
    
    # Normalizar datos / Garantiza que la distribución de entradas sea homogénea, mejorando la convergencia de la red.
    X_train = scaler.fit_transform(X_train) #calcula medias μj y desviaciones σj en cada característica j, y aplica media de xj = (xj - μj) / σj a cada vector de entrenamiento.
    X_test = scaler.transform(X_test) # usa las mismas medias y desviaciones calculadas en el entrenamiento para normalizar el conjunto de test.
    
    # Crear y entrenar modelo
    num_classes = len(set(y_encoded)) #número de clases distintas (tamaño de la salida softmax).
    set_message(f"Entrenando modelo con {num_classes} clases...", 2)
    
    #creacion de la red neuronal
    model = create_neural_network(X_train.shape[1], num_classes)
    """ o	Llama a create_neural_network(input_shape=X_train.shape[1], num_classes).
        o	Define la arquitectura multicapa con regularización, normalización y dropout.
        """
    
    #Callbacks para mejorar el entrenamiento
    callbacks = [ 
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True), #detiene el entrenamiento si la pérdida de validación (val_loss) no mejora tras 10 épocas, y restaura los pesos de la mejor época.
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001) #reduce la tasa de aprendizaje a la mitad (factor=0.5) si val_loss no mejora tras 5 épocas, con un mínimo de 1e-4.
    ]
    
    start_time = time.time() #	Guarda el timestamp de inicio para calcular luego la duración total del entrenamiento.
    
    history = model.fit( # Entrenamiento del modelo
        X_train, y_train, # X_train: matriz de entrenamiento, y_train: etiquetas codificadas.
        epochs=50, #número máximo de pasadas completas sobre el set de entrenamiento.
        batch_size=32, #cada actualización de pesos se hace con 32 muestras.
        validation_split=0.2, #reserva un 20% de X_train para validación interna en cada época.
        callbacks=callbacks, #detiene o ajusta lr según el rendimiento en validación.
        verbose=1 #muestra barra de progreso y métricas por época.
    )
    
    training_time = time.time() - start_time #o	Resta el timepo actual con start_time para obtener segundos totales de entrenamiento.
    
    # Evaluar modelo
    y_pred = np.argmax(model.predict(X_test), axis=1) #devuelve un array (N_test, num_classes) de probabilidades. selecciona la clase con mayor probabilidad para cada muestra.
    accuracy = accuracy_score(y_test, y_pred) #calcula la proporción de predicciones correctas.
    
    # Guardar métricas 
    metrics['accuracy'] = accuracy #precisión en prueba.
    metrics['val_accuracy'] = max(history.history['val_accuracy']) #mejor precisión observada en validación interna (history.history['val_accuracy']).
    metrics['training_time'] = training_time #duración total.
    
    # Guardar modelo y preprocesadores
    model.save(model_file) #guarda arquitectura y pesos en HDF5.
    with open(scaler_file, 'wb') as f:
        pickle.dump(scaler, f) #persiste el objeto StandardScaler.
    with open(encoder_file, 'wb') as f:
        pickle.dump(label_encoder, f)#persiste el LabelEncoder.
    #Estos archivos permitirán luego cargar el pipeline completo sin reentrenar.
    
    set_message(f"Modelo entrenado con precisión: {accuracy:.2%}", 3) #o	Muestra un mensaje final con la precisión de prueba (p. ej. “Modelo entrenado con precisión: 94.50%”).
    is_trained = True
    
    return True


🔍 Resumen de la lógica
1.	Verifica que haya datos suficientes.
2.	Construye matrices NumPy de datos y etiquetas.
3.	Codifica etiquetas a enteros.
4.	Divide en entrenamiento y prueba de forma estratificada.
5.	Normaliza entradas según StandardScaler.
6.	Crea la red neuronal con arquitectura definida.
7.	Entrena con validación interna, early stopping y reducción dinámica del learning rate.
8.	Evalúa en datos de prueba externos y calcula precisión.
9.	Guarda el modelo y los preprocesadores para uso futuro.
10.	Informa al usuario y actualiza el estado interno.


### CARGA DEL MODELO ENTRENADO

In [ ]:
def load_saved_model(): #Define la función encargada de cargar desde disco el modelo entrenado y los preprocesadores.
    global scaler, label_encoder
    try:
        model = load_model(model_file) #Utiliza la función de Keras load_model para leer el archivo HDF5 (hand_gesture_nn_model_v15.h5) y reconstruir la red neuronal completa con sus pesos.
        with open(scaler_file, 'rb') as f: #Abre el archivo que contiene el objeto StandardScaler serializado (hand_gesture_scaler_v15.pkl) en modo lectura binaria.
            scaler = pickle.load(f) #Deserializa el StandardScaler con las medias y desviaciones aprendidas en el entrenamiento, y lo asigna a la variable global scaler.
        with open(encoder_file, 'rb') as f: #Abre el archivo del LabelEncoder (hand_gesture_encoder_v15.pkl).
            label_encoder = pickle.load(f) #Deserializa el encoder que mapea índices numéricos a etiquetas de texto, y lo asigna a la variable global label_encoder.
        set_message("Modelo cargado correctamente", 2)
        return model
    except Exception as e:
        print(f"Error al cargar el modelo: {e}")
        set_message("Error al cargar el modelo", 2)
        return None


### MODELO DE TFLITE

In [ ]:
def predict_tflite(landmarks, tflite_model, scaler, label_encoder, threshold=0.5): #que toma un vector de landmarks y devuelve la etiqueta predicha junto con la confianza, usando el modelo TFLite y los preprocesadores
    """Declara la función que realiza la inferencia.
        o	landmarks: lista o array 1D de características (coordenadas, distancias, ángulos).
        o	tflite_model: instancia de la clase TFLiteModel preparada para inferir.
        o	scaler: objeto StandardScaler ya ajustado.
        o	label_encoder: objeto LabelEncoder con el mapeo inverso.
        o	threshold=0.5: confianza mínima para aceptar una predicción; por debajo se marca como desconocido.
        """
    
    try:
        # Preprocesar los landmarks
        landmarks_array = np.array(landmarks).reshape(1, -1) #añade la dimensión de batch; si landmarks tenía longitud F, ahora es (1, F).
        landmarks_scaled = scaler.transform(landmarks_array) #Normalizar usando el StandardScaler aplica media de xj = (xj - μj) / σj, transformando la muestra individual al mismo espacio que el entrenamiento.
        
        # Realizar predicción
        predictions = tflite_model.predict(landmarks_scaled)[0] #devuelve un array (1, C) de probabilidades para cada clase.	[0] extrae el vector de tamaño C
        
        # Obtener la clase con mayor probabilidad 
        max_prob_idx = np.argmax(predictions) #índice de la probabilidad máxima
        confidence = predictions[max_prob_idx] #el valor de esa probabilidad, entre 0 y 1.
        

        #Umbral de confianza:
        if confidence >= threshold:
            # Decodificar la etiqueta
            predicted_label = label_encoder.inverse_transform([max_prob_idx])[0]
            return predicted_label, confidence
            """ o	Solo aceptamos la predicción si la probabilidad supera threshold (por defecto 0.5).
                o	Esto evita emisiones erráticas cuando el modelo no está seguro. 

               8.	Decodificar índice a texto:
                o	inverse_transform([max_prob_idx]) convierte el índice de clase en la cadena de la seña (ej. "HOLA").
                o	La función retorna una tupla (etiqueta, confianza). """  
        else:
            return "Desconocido", confidence #Si la confianza es baja, devolvemos "Desconocido" y la confianza obtenida. Esto permite manejar gestos no entrenados o detecciones dudosas
    except Exception as e:
        print(f"Error en la predicción: {e}")
        return "Error", 0.0


¿Cómo encaja esta función?
•	Se llama en el bucle de inferencia en vivo (me_15_v6_evaluacion.ipynb) tras extraer y normalizar los landmarks.
•	Devuelve un string con la seña detectada (o "Desconocido"/"Error") y su nivel de confianza.
•	Con esa información se decide si reproducir la salida por audio y mostrar el texto en pantalla.


### CONVERSION A TFLITE

In [ ]:
def convert_to_tflite(model_file, model_tflite): #toma un modelo Keras y lo convierte a formato TensorFlow Lite
    """ o	model_file: ruta al archivo HDF5 del modelo Keras entrenado.
        o	model_tflite: ruta de salida donde se guardará el .tflite.
        """
    try:
        if not os.path.exists(model_file):
            raise FileNotFoundError(f"El archivo {model_file} no existe.")
        
        # Cargar el modelo entrenado
        modelo = load_model(model_file)
        """Carga del modelo Keras
        o	Usa load_model de Keras para leer model_file y reconstruir la arquitectura y pesos.
        o	Se asigna a la variable local modelo.
        """
        
        # Convertir a TensorFlow Lite
        converter = tf.lite.TFLiteConverter.from_keras_model(modelo) #Crea un objeto TFLiteConverter a partir del modelo Keras cargado.
        tflite_model = converter.convert() #Llama a converter.convert(), que traduce la red y sus pesos al formato optimizado de TensorFlow Lite, devolviendo un bytes con el modelo TFLite
        
        # Guardar el modelo convertido
        with open(model_tflite, "wb") as f:
            f.write(tflite_model)
        
        print("Modelo convertido a TensorFlow Lite.")
    except Exception as e:
        print("Error al convertir el modelo a TFLite:", e)

### ELIMINADOR DE SEÑAS

In [ ]:
def delete_gesture(target_label): #Declara la función que recibe target_label, la etiqueta (nombre de seña) que se desea eliminar.
    global data, labels
    if target_label not in labels: #Comprueba si la etiqueta existe en la lista de etiquetas.
        print(f"Error: La etiqueta '{target_label}' no existe") #Si no se encuentra, muestra un mensaje de error y retorna False.
        return False
    
    # Filtrar elementos a mantener
    new_data = [] #Lista temporal para almacenar los vectores de características que no corresponden a la etiqueta a eliminar
    new_labels = [] #Lista temporal para almacenar las etiquetas que no corresponden a la etiqueta a eliminar
    deleted_count = 0 #Contador de cuántas muestras se eliminaron
    
    for feature, label in zip(data, labels): #Itera sobre las listas data y labels simultáneamente, emparejando cada vector de características con su etiqueta correspondiente.
        if label == target_label: #Si la etiqueta coincide con la que se desea eliminar:
            deleted_count += 1 # Incrementa el contador de eliminaciones
        else:
            new_data.append(feature) # Agrega el vector de características a la nueva lista si no es la etiqueta objetivo
            new_labels.append(label) # Agrega la etiqueta a la nueva lista si no es la etiqueta objetivo
    
    # Actualizar listas globales
    data.clear() # Limpia las listas globales data y labels para evitar duplicados o residuos de la eliminación.
    labels.clear() # Limpia las etiquetas globales
    data.extend(new_data) # Agrega los vectores de características filtrados a la lista global data
    labels.extend(new_labels) # Agrega las etiquetas filtradas a la lista global labels
    
    print(f"Se eliminaron {deleted_count} muestras de '{target_label}'")
    return True

### GENERADOR DE AUDIOS

In [ ]:
# Configuración de directorios y archivos
audio_dir = "pyttsx3_audios" #carpeta donde se guardarán los audios .wav generados.
os.makedirs(audio_dir, exist_ok=True)

# Configurar motor TTS
engine = pyttsx3.init() #inicializa el motor de Text-To-Speech (síntesis de voz offline).
engine.setProperty('rate', 150) #velocidad de habla (palabras por minuto).
engine.setProperty('volume', 1.0)  # volumen máximo (rango de 0.0 a 1.0).

def trim_audio_silence(file_path):
    """Recorta silencios al inicio y final del audio"""
    audio = AudioSegment.from_file(file_path, format="wav")

    # Parámetros ajustables
    config = {
        'min_silence_len': 200,     # 200 ms de silencio mínimo para considerar corte
        'silence_thresh': -45,      # -45 dB de umbral de silencio
        'end_buffer': 150           # 150 ms extra al final
    }
    
    # Detectar segmentos no silenciosos
    nonsilent_parts = detect_nonsilent(
        audio,
        min_silence_len=config['min_silence_len'], # Duración mínima de silencio a considerar (ms)
        silence_thresh=config['silence_thresh'] # Umbral de volumen para considerar silencio (dB)
    )
    
    if nonsilent_parts:
        start = max(0, nonsilent_parts[0][0] - 50)  # 50 ms buffer inicial
        end = nonsilent_parts[-1][1] + config['end_buffer']
        trimmed_audio = audio[start:end]

        # --- AÑADIDO: subir el volumen  ---
        gain_db = 24  # ganancia en db
        trimmed_audio = trimmed_audio + gain_db

        trimmed_audio.export(file_path, format="wav")

def compilador_audios(label):
    """Genera y ajusta audio para eliminar silencios"""
    nombre_archivo = label.replace(' ', '_').lower() + '.wav'
    ruta_audio = os.path.join(audio_dir, nombre_archivo)
    
    if os.path.exists(ruta_audio):
        return
    
    temp_path = os.path.join(audio_dir, "temp.wav")
    try:
        # Generar audio temporal
        engine.save_to_file(label, temp_path)
        engine.runAndWait()
        
        # Recortar y renombrar
        trim_audio_silence(temp_path)
        os.rename(temp_path, ruta_audio)
        print(f"Audio generado: {nombre_archivo}")
        
    except Exception as e:
        print(f"Error generando {label}: {str(e)}")
        if os.path.exists(temp_path):
            os.remove(temp_path)

def generar_audios():
    """Genera audios para todas las etiquetas únicas"""
    etiquetas_unicas = set(labels)
    print("\nGenerando audios para señas...")
    
    for label in etiquetas_unicas:
        compilador_audios(label)
    
    print("Proceso de generación de audios completado\n")


### MENU

In [54]:
def print_menu():
    print("\n=== MENU PRINCIPAL ===")
    print("1. Recolectar nueva seña")
    print("2. Entrenar modelo")
    print("3. Listar señas cargadas")
    print("4. Eliminar señas")
    print("5. Generar Audios")
    print("0. Salir")


### LISTADO DE GESTOS

In [55]:
def list_gestures():
    # Asumiendo que 'labels' es la lista donde se guardan las señas
    if not labels:
        print("No hay señas guardadas.")
    else:
        unique_gestures = list(set(labels))
        print("\n--- Señas Guardadas ---")
        for i, gesture in enumerate(unique_gestures, 1):
            print(f"{i}. {gesture}")


### RECOLECCION DE SEÑAS

In [ ]:
def run_collection_mode(): #1.	Define la función principal que se ejecuta cuando el sistema entra en modo "recolectar nuevas señas".
    # Inicia la cámara
    try:
        cap = UDPCamera() #Intenta instanciar el objeto UDPCamera, que recibe frames de video vía UDP desde la Raspberry Pi.
        print("Cámara UDP iniciada para recolección.")
    except Exception as e:
        print(f"Error al iniciar la cámara: {str(e)}")
        return
    
    while is_collecting:  # Este bucle se mantiene activo mientras is_collecting == True, lo cual se establece al llamar a start_collection().
        ret, frame = cap.read() #Intenta capturar un frame de video desde el socket UDP.
        
        if not ret: #Si no se pudo capturar un frame, imprime un mensaje de error y espera 100 ms antes de reintentar.
            time.sleep(0.1)
            continue

        landmarks, hands_detected = extract_hand_landmarks(frame, send_sock)
        """8.	Llama a la función de detección y extracción de puntos de mano.
                o	landmarks: vector numérico de características.
                o	hands_detected: indica si hay al menos una mano en el frame.
                o	send_sock: permite enviar coordenadas de muñeca vía UDP a la Raspberry.
                """
        frame_h, frame_w, _ = frame.shape #Extrae alto y ancho de la imagen para colocar textos y gráficos en pantalla.

        # Mostrar información en pantalla durante la recolección
        progress = int((samples_collected / max_samples) * frame_w)
        cv2.rectangle(frame, (0, 0), (progress, 20), (0, 255, 0), -1) #Calcula el progreso de muestras recolectadas y dibuja una barra verde en la parte superior de la imagen.

        cv2.putText(frame, f"Recolectando: {current_gesture} ({samples_collected}/{max_samples})", 
                    (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2) #Muestra en pantalla el nombre de la seña y la cantidad de muestras recolectadas hasta el momento.
       
       
        #Detección de manos y recolección
        if hands_detected: #Si hay manos en el frame, se intenta capturar la muestra actual (pasando los landmarks a collect_sample()).
            collect_sample(landmarks)
        else:
            cv2.putText(frame, "¡Muestra las manos!", (frame_w//2 - 100, frame_h//2), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
        if not is_collecting:  # Cuando termina la recolección
            menu_active = True
            save_data()
            """Si en algún momento is_collecting se pone en False (por ejemplo, al completar las max_samples), 
            se activa el menú principal (menu_active) y se guardan los datos recolectados en disco."""
        
        cv2.imshow("Recolectar Señas", frame) #Muestra el frame con superposición de información y gráficos en una ventana de OpenCV.
        
        key = cv2.waitKey(1)
        # Pulsar esc para salir de la recolección.
        if key == 27:
            break
    
    cap.release()
    cv2.destroyAllWindows()

¿Qué hace esta función?
•	Muestra en tiempo real los frames con indicaciones.
•	Captura muestras automáticamente mientras detecta una mano.
•	Guía visualmente al usuario en la recolección.
•	Guarda todo al completar o al presionar ESC.


### FUNCION PRINCIPAL 

Esta es la interfaz central donde el usuario puede recolectar, entrenar, gestionar y generar audios para el sistema de reconocimiento de gestos.

In [57]:
def main():
    global model, is_trained, data, labels
    
    # Inicialización del sistema
    is_trained = False
    model = None
    data = []
    labels = []
    
    # Cargar datos existentes
    load_data()
    
    # Intentar cargar modelo si existe
    if check_model_exists():
        model = load_saved_model()
        is_trained = True
    else:
        is_trained = False

    # Mostrar el menú en la consola
    print_menu()

    # Bucle principal de selección en consola
    while True:
        opcion = input("\nSelecciona una opción (Recolectar: 1, Entrenar: 2, Listar: 3, Eliminar: 4, Gen.Audio: 5, Salir: 0): ").strip()
        
        if opcion == '1':
            # Recolección de señas
            gesture_name = input("Ingrese nombre de la seña (ej. 'Hola'): ")
            if gesture_name:
                start_collection(gesture_name)
                # Iniciar la cámara para mostrar video durante la recolección
                run_collection_mode()
                
        elif opcion == '2':
            if len(data) > 10:
                train_model()
                model = load_saved_model() if check_model_exists() else None
                is_trained = True
                print("Entrenamiento completado. Modelo entrenado.")
                convert_to_tflite(model_file, model_tflite)
                print("Convertido a TFLite para evaluación en tiempo real")
            else:
                print("¡Necesitas al menos 10 muestras para entrenar!")
                
        elif opcion == '3':
            list_gestures()  # Lista las señas cargadas

        elif opcion == '4':
            if not labels:
                print("No hay señas guardadas para eliminar")
                exit()
            print("\n--- Señas Registradas ---")
            for label in set(labels):
                print(f"- {label}")
            target_label = input("\nIngrese el nombre exacto de la seña a eliminar: ").strip()
            if delete_gesture(target_label):
                save_data()
            else:
                print("No se realizaron cambios en los datos")
        
        elif opcion == '5':
            if not labels:
                print("No hay señas guardadas")
                exit()
            # Generar audios automáticamente
            generar_audios()
            

                
        elif opcion == '0':
            print("Saliendo del programa...")
            break
        else:
            print("Opción inválida, intenta nuevamente.")
        
        # Mostrar nuevamente el menú luego de finalizar la opción seleccionada.
        print_menu()


# EJECUTAR PROGRAMA

In [58]:
if __name__ == "__main__":
    main()
    save_data()  # Guarda los datos recolectados


=== MENU PRINCIPAL ===
1. Recolectar nueva seña
2. Entrenar modelo
3. Listar señas cargadas
4. Eliminar señas
5. Generar Audios
0. Salir

Generando audios para señas...
Audio generado: abecedario.wav
Proceso de generación de audios completado


=== MENU PRINCIPAL ===
1. Recolectar nueva seña
2. Entrenar modelo
3. Listar señas cargadas
4. Eliminar señas
5. Generar Audios
0. Salir
Saliendo del programa...
